In [102]:
import pandas as pd 
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import json 
import os
from tqdm import tqdm
import sqlite3

In [ ]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [2]:
ccm=SpotifyClientCredentials(client_id= credentials['client_id'],
    client_secret= credentials['client_secret'])

spotify = spotipy.Spotify(client_credentials_manager=ccm)

In [3]:
featured_playlists = spotify.featured_playlists()['playlists']['items']#need to get the names and then the id's of the playlist. 
names_of_playlists = [names['name'] for names in featured_playlists]
uri_of_playlists = [uri['uri'] for uri in featured_playlists]


#this creates a dictionary of the 
artist_genres = dict(zip(names_of_playlists, uri_of_playlists))
list_of = list(artist_genres.values())

In [4]:
def playlist_info(uri):
    # Retrieves the playlist and extracts artist URIs
    results = spotify.playlist(uri)

    with open(f"./raw_data/playlist/{results['name']}.json", "w") as outfile: 
        json.dump(results, outfile)


def load_files(folder_path):
    json_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.json')]


    raw_data_list = []
    for file_path in json_files:
        with open(file_path, 'r') as file:
            raw_data_list.append(json.load(file))
    return raw_data_list

def get_artist_ids(data): 
        artist_ids = [track['track']['artists'][0]['uri'] for track in data['tracks']['items']]

        
        #_artist_info = [spotify.artist(artist) for artist in artist_ids]
      
        #with open(f"./raw_data/genres/{_artist_info['names']}.json", "w") as outfile: 
            #json.dump(_artist_info, outfile)

        #this is a list containing the id's of the artists 
        return list(set(artist_ids))

def get_info(artist_ids):
     #we will have to loop through the list before calling this function so it applies to each playlist. 
     return [spotify.artist(artist)['genres'] for artist in artist_ids]


    

In [8]:
master_list = [playlist_info(playlist) for playlist in list_of]

data_files = load_files('./raw_data/playlist')

artist_ids = []
for i in tqdm(data_files): 
    artist_ids.append(get_artist_ids(i))




100%|██████████| 20/20 [00:00<00:00, 20510.04it/s]


In [9]:
list_of_genres = []
for i in tqdm(artist_ids):
    list_of_genres.append(get_info(i))

100%|██████████| 20/20 [01:23<00:00,  4.17s/it]


In [ ]:
#I understand now, the raw data in the playlist category is always going to give it in alphebtical order no matter what. The index of the actual datframe is differen . 

In [34]:
#creates a list that finds the unique cateogies in the list. 
new_list = []
for i in list_of_genres:
    new_list.append(list(set([genre for genre_list in i for genre in genre_list])))





#now it times to create the dictionary and then upload it to a sql database. 
new_list[0]

['color noise', 'water', 'white noise', 'rain', 'environmental', 'binaural']

In [32]:
names_of_playlists


['Hot Hits UK',
 'Today’s Top Hits',
 'This Is Taylor Swift',
 'Summer Dance Hits 2024',
 'Sleep',
 'Disney Hits',
 'All Out 80s',
 'Massive Dance Hits',
 'White Noise 10 Hours',
 'New Music Friday UK',
 'Summer Bangers',
 'This Is Eminem',
 'Rap UK',
 'Country Top 50',
 'All Out 2000s',
 'Taylor Swift | The Eras Tour Official Setlist',
 'Massive Drum & Bass',
 'RapCaviar',
 'This Is Coldplay',
 'phonk']

In [56]:

#creates a dictionary with the name of th eplaylist and its id
fixed_dict = {data_names['name']: data_names['uri'] for data_names in data_files}


#getting the followers of all the playlists

number_of_followers = [followers['followers']['total'] for followers in data_files]






In [84]:




playlist_names = list(fixed_dict.keys())
list_of_uri = list(fixed_dict.values())



playlist_category = dict(zip(playlist_names, new_list))






playlist = list(fixed_dict.keys())

playlist


['White Noise 10 Hours',
 'Rap UK',
 'Massive Dance Hits',
 'Summer Bangers',
 'This Is Coldplay',
 'Country Top 50',
 'phonk',
 'Sleep',
 'All Out 80s',
 'Taylor Swift | The Eras Tour Official Setlist',
 'New Music Friday UK',
 'Today’s Top Hits',
 'Disney Hits',
 'All Out 2000s',
 'Massive Drum & Bass',
 'Hot Hits UK',
 'This Is Taylor Swift',
 'This Is Eminem',
 'RapCaviar',
 'Summer Dance Hits 2024']

In [94]:
#create dataframe playlist name uri and followers

df_playlist = pd.DataFrame({'playlist_name': playlist, 'uri': list_of_uri, 'followers': number_of_followers}).reset_index()

df_playlist = df_playlist.rename(columns={'index': 'id'})
df_playlist['id'] = df_playlist['id'].apply(lambda x: x+1)
df_playlist


,id,playlist_name,uri,followers
0,1,White Noise 10 Hours,spotify:playlist:37i9dQZF1DWUZ5bk6qqDSy,1434173
1,2,Rap UK,spotify:playlist:37i9dQZF1DX6PKX5dyBKeq,1383911
2,3,Massive Dance Hits,spotify:playlist:37i9dQZF1DX5uokaTN4FTR,1400555
3,4,Summer Bangers,spotify:playlist:37i9dQZF1DWWevrQ6HpxHI,277696
4,5,This Is Coldplay,spotify:playlist:37i9dQZF1DXaQm3ZVg9Z2X,3950474
5,6,Country Top 50,spotify:playlist:37i9dQZF1DX7aUUBCKwo4Y,1426991
6,7,phonk,spotify:playlist:37i9dQZF1DWWY64wDtewQt,8588221
7,8,Sleep,spotify:playlist:37i9dQZF1DWZd79rJ6a7lp,5942707
8,9,All Out 80s,spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe,11325468
9,10,Taylor Swift | The Eras Tour Official Setlist,spotify:playlist:37i9dQZF1DX0D996ZXujBy,491240


In [140]:
playlist_category = pd.DataFrame({'categories':new_list}).reset_index()
playlist_category = playlist_category.rename(columns = {'index': 'playlist_id'})


playlist_category['playlist_id'] = playlist_category['playlist_id'].apply(lambda x: x+1)

playlist_category = playlist_category.explode('categories')

In [141]:
playlist_category

,playlist_id,categories
0,1,color noise
0,1,water
0,1,white noise
0,1,rain
0,1,environmental
...,...,...
19,20,slap house
19,20,stutter house
19,20,neo-synthpop
19,20,new french touch


In [105]:
%reload_ext sql
%config SqlMagic.autocommit=True

In [119]:
%sql sqlite:///data/playlist_uk.db --alias playlist_uk

Connecting to 'playlist_uk'

In [123]:
DATA_FOLDER = os.path.join('data/')

In [124]:

conn = sqlite3.connect(os.path.join(DATA_FOLDER, 'playlist_uk.db'))

In [122]:
#creaitng the tables




#dataframes are already defined so simply just need to create the tables. 




'//data/playlist_uk.db'

In [144]:
%%sql

    -- creating the tables in order for analysis 

DROP TABLE IF EXISTS playlist_name; 

CREATE TABLE playlist_name (
 "id" INTEGER PRIMARY KEY,
 "playlist_name" VARCHAR(50),
 "uri" VARCHAR(50),
 "Followers" INTEGER
);

DROP TABLE IF EXISTS playlist_category; 

CREATE TABLE playlist_category (

    "playlist_id" INTEGER,
    "categories" VARCHAR(50),
    FOREIGN KEY("playlist_id") REFERENCES "playlist_name" ('id')
); 

Running query in 'playlist_uk'

++
||
++
++

In [145]:
#uploading data
df_playlist.to_sql('playlist_name', conn, if_exists='append', index=False)

playlist_category.to_sql('playlist_category', conn, if_exists='append', index=False)

772

Running query in 'playlist_uk'

playlist_id,categories
1,color noise
1,water
1,white noise
1,rain
1,environmental
1,binaural
2,birmingham hip hop
2,afroswing
2,conscious hip hop
2,bronx drill
